# Informed search - the A* algorithm

The depth-first search and breadth-first search considered in the previous lesson are completely *blind* algorithms: they're only concerned whether the currently considered state is a goal state or not. They're unable to distinguish whether a state is easy or hard to reach, or whether it is near or far of the goal. This makes them very inefficient search algorithms. To allievate the issue, we introduce informed search algorithms. The information is given to an algorithm in two ways:

1. By using an *action cost* function $c(s,a)$, which, given a state $s$ and an action $a$ available in this state, returns its cost as a non-negative number.
2. By using a *heuristic* $h(s)$, which, given a state, estimates the lowest cost to reach a goal state from the given state.

Given a sequence of actions $a_1, \ldots, a_n$ and an initial state $s_1$, we can express the *total cost* of reaching the state $s_{n+1}$ by executing the sequence as:
$$ c(s_1, a_1, \ldots, a_{n-1}) = \sum_{i=1}^n c(s_i, a_i) $$
and the *expected cost* of the solution as the sum of the total cost and the estimate cost of reaching the goal from the state $s_{n+1}$
$$ f(s_1, a_1, \ldots, a_n) = c(s_1, a_1, \ldots, a_n) + h(s_{n+1}) $$

The heuristic function is a bit tricky, because we want it to have two properties:
* *Admissibility*: It must never *overestimate* the true cost of reaching the goal. 
* *Consistency*: Let $s$ be a state such that $a$ is an available action in this state and $s'$ is the state reached by executing this action. The heuristic should fulfil triangle inequality, that is, the estimated cost to reach the goal from $s$ should be no greater than the cost of executing the action $a$ + the estimated cost of reaching the goal from the new state.
$$ h(s) \leq c(s, a) + h(s') $$

One can prove that admissibility follows from consistency, but consistency is important only if there are multiple paths to reach the same state (i.e., we are searching in a graph, not in a tree). Otherwise, admissability is sufficient.

Lets extend the class `Problem` from the previous lesson with two new functions `action_cost` and `heuristic`, which correspond to the functions $c(s,a)$ and $h(s)$ described above.

In [45]:
from abc import ABC, abstractmethod

class Problem(ABC):

    @property
    @abstractmethod
    def initial_state(self):
        raise NotImplementedError

    @abstractmethod
    def available_actions(self, state):
        raise NotImplementedError

    @abstractmethod
    def take_action(self, state, action):
        raise NotImplementedError

    @abstractmethod
    def is_goal(self, state) -> bool:
        raise NotImplementedError

    @abstractmethod
    def action_cost(self, state, action) -> float:
        raise NotImplementedError

    @abstractmethod
    def heuristic(self, state) -> float:
        raise NotImplementedError

To make a concrete example, lets revisit the vacuum world. 

![](aima-images/fig2_2.png)

Below, we assume a very simple model:
* Any action costs 1. This corresponds to searching for the shortest plan.
* The heuristic estimation is the number of fields which are still dirty. 


Lets consider the properties of the heuristic:
* Is is admissible? The heuristic value is equal to the number of 'Suck' actions that are yet to be executed and ignores the spatial aspect (i.e., moving between the rooms), thus never overestimating.
* Is it consistent? As a consequence of a single action the heuristic value can decrease by at most 1 (if the action happens to be 'Suck' and the room is dirty). The cost of any action is 1, so rewriting the triangle inequality we arrive at:
$$ h(s) \leq c(s, a) + h(s') = \begin{cases} 1 + (h(s)-1) & a=\text{'Suck' and the room was dirty} \\ 1 + h(s) & \text{otherwise} \end{cases} $$
* Is it the best we could have? By no means! We could include the spatial aspect.

In [46]:
class VacuumProblem(Problem):
    @property
    def initial_state(self):
        return (0, (True, True))
    
    def available_actions(self, state):
        return ["Left", "Suck", "Right"]
        
    def take_action(self, state, action):
        robot, dirty = state
        if action == "Left":
            return (max(robot-1, 0), dirty)
        elif action == "Suck":
            new_dirty = list(dirty)
            new_dirty[robot] = False
            return (robot, tuple(new_dirty))
        elif action == "Right":
            return (min(robot+1, len(dirty)-1), dirty)        
        raise Exception('Invalid action')
    
    def is_goal(self, state) -> bool:
        return not any(state[1])
    
    def action_cost(self, state, action):
        return 1
    
    def heuristic(self, state):
        return sum(state[1])

## Task 1: Implement the A* algorithm

To implement the A* algorithm you must have a **priority queue**. Luckily, Python comes with one, so you don't need to implement it by yourself. Then, the algorithm is very simple: 
1. Start with a queue containing a single item - the initial state
2. Repeat until the queue is not empty:
   1. Pick an item with the lowest expected cost
   2. If this is the goal, return the sequence of actions necessary to reach this state
   3. Otherwise, for each available action, create a new entry in the queue corresponding to the state reached after executing the action.

Keep track of visited states - it may be necessary to discover that the problem has no solutions. Assume states are hashable.
  
In the cell below implement the algorithm in a similar manner as the BFS and DFS in the previous lesson: the sole argument is an object of the class Problem and the function should return a list of actions to achieve a goal state from the initial state.
If it is impossible to reach the goal, return `None`.
Count the number of states visited during the search (i.e., the number of times you removed an element from the queue) and print in out before returning from the function, it will be useful later on to compare different heuristics.

In [47]:
import heapq
def astar(problem: Problem):
    frontier = []
    visited = set()
    visited_count = 0
    initial_state = problem.initial_state
    g_cost = 0
    h_cost = problem.heuristic(initial_state)
    f_cost = g_cost + h_cost
    initial_path = []
    heapq.heappush(frontier, (f_cost, g_cost, initial_state, initial_path))
    while frontier:
        f_cost, g_cost, current_state, current_path = heapq.heappop(frontier)
        visited_count += 1
        if problem.is_goal(current_state):
            print(f"Nodes visited during search: {visited_count}")
            return current_path
        if current_state in visited:
            continue
        visited.add(current_state)
        for action in problem.available_actions(current_state):
            new_state = problem.take_action(current_state, action)
            if new_state not in visited:
                new_g_cost = g_cost + problem.action_cost(current_state, action)
                new_h_cost = problem.heuristic(new_state)
                new_f_cost = new_g_cost + new_h_cost
                new_path = current_path + [action]
                heapq.heappush(frontier, (new_f_cost, new_g_cost, new_state, new_path))

    print(f"Nodes visited during search: {visited_count}")
    return None

Now lets test your code in the vacuum world!

In [48]:
astar(VacuumProblem())

Nodes visited during search: 6


['Suck', 'Right', 'Suck']

## Task 2: Variants of the vacuum world

Now lets consider a different take on the vacuum world in which the heuristic is not admissible and increases as the number of dirty fields decreases.

In [49]:
class VacuumProblem1(VacuumProblem):
    def action_cost(self, state, action):
        return 1

    def heuristic(self, state):
        return len(state[1]) - sum(state[1])

astar(VacuumProblem1())

Nodes visited during search: 7


['Suck', 'Right', 'Suck']

And another in which heuristic grossly overestimates the cost.

In [50]:
class VacuumProblem2(VacuumProblem):
    def action_cost(self, state, action):
        return 1

    def heuristic(self, state):
        return 10 * sum(state[1])

astar(VacuumProblem2())

Nodes visited during search: 4


['Suck', 'Right', 'Suck']

**Which of the three heuristic functions (`VacuumProblem`, `VacuumProblem1`, `VacuumProblem2`) is the best? Is it the expected answer given the properties of the heuristics? If not, explain why an unorthodox approach works better.**

**Answer**

Based on the principles of A* search, the VacuumProblem heuristic (counting dirty rooms) is the best because it is the only admissible heuristic. An admissible heuristic never overestimates the true cost (meaning h(n) <= h*(n)), which is the non-negotiable requirement for A* to guarantee finding the optimal (shortest) path. Both VacuumProblem1 (counting clean rooms) and VacuumProblem2 (10 * dirty rooms) are inadmissible as they overestimate the cost, thus voiding any guarantee of optimality.

This answer is initially unexpected because the inadmissible VacuumProblem2 found the optimal solution while visiting the fewest nodes (4, vs. 6 for the "best" admissible heuristic). This unorthodox approach "worked" better only by coincidence. Its heuristic value was so large that it effectively overshadowed the true path cost (g(n)), turning A* into a Greedy Best-First Search. In this simple problem, the greedy path happened to be the optimal one. In a more complex scenario, this heuristic would almost certainly have returned a suboptimal path. Therefore, VacuumProblem remains the only correct and reliable choice.

## Task 3: 8-puzzle problem

Recall the 8-puzzle problem from the previous lesson. Reuse your code and implement an extended version assuming that each action costs 1. Propose 3 (at least) admissible heuristics. This time **don't change the initial state (nor the goal state)**, your solution should be capable enough to solve this. The classes are named `PuzzleProblem1`, `PuzzleProblem2`, `PuzzleProblem3` - remember not to rename them!

![](aima-images/fig3_4.png)

In [51]:
class PuzzleProblem1(Problem):
    def __init__(self):
        self.goal = ((0, 1, 2),
                     (3, 4, 5),
                     (6, 7, 8))
    def _print_board(self, state, title):
        print(title)
        for r in range(3):
            row_str = " ".join(str(tile) if tile != 0 else "0" for tile in state[r])
            print(f"  [{row_str}]")
        print()
    def display_problem(self):
        print("8-Puzzle Problem")
        print("")
        self._print_board(self.initial_state, "Start State:")
        self._print_board(self.goal, "Goal State:")
        print("")
    @property
    def initial_state(self):
        return ((7, 2, 4),
                (5, 0, 6),
                (8, 3, 1))
    def _find_blank(self, state):
        for r in range(3):
            for c in range(3):
                if state[r][c] == 0:
                    return r, c
        return None
    def available_actions(self, state):
        actions = []
        r, c = self._find_blank(state)
        if r > 0:
            actions.append('Up')
        if r < 2:
            actions.append('Down')
        if c > 0:
            actions.append('Left')
        if c < 2:
            actions.append('Right')
        return actions
    def take_action(self, state, action):
        r, c = self._find_blank(state)
        new_state_list = [list(row) for row in state]
        if action == 'Up':
            target_r, target_c = r - 1, c
        elif action == 'Down':
            target_r, target_c = r + 1, c
        elif action == 'Left':
            target_r, target_c = r, c - 1
        elif action == 'Right':
            target_r, target_c = r, c + 1
        new_state_list[r][c] = new_state_list[target_r][target_c]
        new_state_list[target_r][target_c] = 0
        new_state = tuple(tuple(row) for row in new_state_list)
        return new_state
    def is_goal(self, state) -> bool:
        return state == self.goal
    def action_cost(self, state, action) -> float:
        return 1.0
    def heuristic(self, state) -> float:
        misplaced_count = 0
        for r in range(3):
            for c in range(3):
                tile = state[r][c]
                if tile == 0:
                    continue
                if tile != self.goal[r][c]:
                    misplaced_count += 1
        return float(misplaced_count)

In [52]:
# I add this to see the output (It is better when you can see the results)
p1 = PuzzleProblem1()

p1.display_problem()

8-Puzzle Problem

Start State:
  [7 2 4]
  [5 0 6]
  [8 3 1]

Goal State:
  [0 1 2]
  [3 4 5]
  [6 7 8]




**Prove that this heuristic is admissible.**

**Answer**

To be admissible, a heuristic must never overestimate the true cost to reach the goal, meaning the heuristic estimate, h(n), must always be less than or equal to the true optimal cost, h*(n).

Our heuristic, h(n), counts the number of misplaced tiles, let's call this count k. The true cost, h*(n), is the minimum number of moves required to solve the puzzle. Each move in the 8-puzzle consists of swapping the blank tile with an adjacent tile. This action, in the absolute best-case scenario, can only move one misplaced tile into its correct, final position. Therefore, to correct k misplaced tiles, a minimum of k moves are required.

Since the true cost h*(n) must be at least k, and our heuristic h(n) is exactly k, the condition h(n) <= h*(n) is always satisfied. The heuristic is admissible.

In [53]:
class PuzzleProblem2(PuzzleProblem1):

    def heuristic(self, state) -> float:

        total_distance = 0
        for r in range(3):
            for c in range(3):
                tile = state[r][c]

                if tile == 0:
                    continue

                goal_r, goal_c = divmod(tile, 3)

                distance = abs(r - goal_r) + abs(c - goal_c)
                total_distance += distance

        return float(total_distance)

**Prove that this heuristic is admissible.**

**Answer**

To be admissible, the heuristic estimate h(n) must be less than or equal to the true optimal cost h*(n).

Our heuristic, h(n), is the sum of the Manhattan distances for all tiles to their goal positions. The Manhattan distance for a single tile is the minimum number of horizontal and vertical steps required to move it to its goal, assuming no other tiles are in the way.

The true cost, h*(n), is the actual number of moves (swaps with the blank tile) to reach the goal. A single move can only shift one tile one step up, down, left, or right. When a tile is moved, its personal Manhattan distance to its goal can decrease by at most 1 (if it moves closer) or it might even increase (if it's forced to move away). Crucially, one move can reduce the total Manhattan distance sum, h(n), by at most 1.

Since the total distance to reduce is h(n), and each move can reduce this total by at most 1, it must take at least h(n) moves to reach the goal (where the total distance is 0). Therefore, h(n) <= h*(n) is always true and the heuristic is admissible.

In [54]:
class PuzzleProblem3(PuzzleProblem1):

    def heuristic(self, state) -> float:

        return 0.0

**Prove that this heuristic is admissible.**

**Answer**

To be admissible, the heuristic estimate h(n) must be less than or equal to the true optimal cost h*(n) for all states.

In this case, our heuristic is h(n) = 0. The true cost to reach the goal, h*(n), represents the minimum number of moves from the current state to the goal state. The number of moves can never be negative; it is always zero (if already at the goal) or a positive integer.

Therefore, the condition 0 <= h*(n) is always true. Since h(n) is always 0 and h*(n) is always greater than or equal to 0, the rule h(n) <= h*(n) is satisfied for every possible state. The heuristic is admissible.

Run your heuristics on the given problem. **All three plans should be of the same length**, but they may differ.

In [55]:
plan0 = astar(PuzzleProblem1())
plan1 = astar(PuzzleProblem2())
plan2 = astar(PuzzleProblem3())

print("Is plan0==plan1?", plan0 == plan1)
print("Is len(plan0)==len(plan1)?", len(plan0) == len(plan1))
print("Is plan0==plan2?", plan0 == plan2)
print("Is len(plan0)==len(plan2)?", len(plan0) == len(plan2))
print("Is plan1==plan2?", plan1 == plan2)
print("Is len(plan1)==len(plan2)?", len(plan1) == len(plan2))

Nodes visited during search: 50634
Nodes visited during search: 4347
Nodes visited during search: 208490
Is plan0==plan1? True
Is len(plan0)==len(plan1)? True
Is plan0==plan2? True
Is len(plan0)==len(plan2)? True
Is plan1==plan2? True
Is len(plan1)==len(plan2)? True


**Which of the heuristics is the best for this task? Why is that?**

**Answer**

The best heuristic for this task is `PuzzleProblem2` (Manhattan Distance).

All three heuristics are admissible, which is why they all correctly found the same optimal path, as confirmed by the output showing `len(plan0) == len(plan1) == len(plan2)`. However, the "best" heuristic is not just the one that finds the correct path, but the one that finds it most efficiently. The efficiency is measured by the number of nodes visited during the search.

The Manhattan Distance heuristic is the most "informed" of the three. It provides an estimate h(n) that is much closer to the true cost h^*(n) without ever overestimating it. Because its estimate is more accurate, it allows the A\* algorithm to be "smarter" and waste far less time exploring unpromising paths. This is definitively proven by the output: `PuzzleProblem2` (Manhattan) found the solution by visiting only 4,347 nodes, whereas `PuzzleProblem1` (Misplaced Tiles) required 50,634 nodes and `PuzzleProblem3` (h=0) performed a "blind" search, visiting 208,490 nodes. Because it is the most informed admissible heuristic, Manhattan Distance is the most efficient and therefore the best.

------------
The pictures and the description of 8-puzzle are from "Artificial Intelligence: A Modern Approach" 3rd ed.